# imports

In [ ]:
%matplotlib inline
import eelbrain as eel
import numpy as np
import scipy, pathlib, importlib, mne, time, os, sys, statsmodels, statsmodels.stats.multitest, matplotlib
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
mne.set_log_level(verbose='error')
import utils, TRFs
from pathnames import *

colors = [(0, 0.2, 0.7), (0, 0.5, 0.8), (0.4, 0.8, 1), (0.4, 0.9, 0.7)]

preds = {}
preds_a = {}
dBstrs=['66dB', '54dB', '42dB', '30dB']

# preprocess EEG data

In [ ]:
importlib.reload(TRFs)
preprocessed_path.mkdir(exist_ok=True)

for i in range(4, 5):
    subject = f'TP00{i:02d}'
    print(subject)

    if subject == 'TP0009':
        remove_segments = [None, [[2400, 2540]]] # redid some of the speech material
    else:
        remove_segments = None
        
    if subject == 'TP0003':
        eegfiles = [f'{subject}_speech_part1a.bdf', f'{subject}_speech_part1b.bdf', f'{subject}_speech_part2.bdf']
    else:
        eegfiles = [f'{subject}_speech_part1.bdf', f'{subject}_speech_part2.bdf']

    savestr = f'{subject}_speech'
    ch = 'Cz'
    refc = ['EXG1', 'EXG2']
    fsds = 4096
    stdmul = 100
    N = 40
    t1 = 0
    t2 = 62
    save_raw_flag = True

    preprocessed_file = preprocessed_path / f'{savestr}_preprocessed_{ch}_{refc[0]}{refc[1]}.pkl'
    datadict = TRFs.preprocess_channel(savestr, preprocessed_file.stem, eegfiles, rawdata_path, subject, preprocessed_path, ch=ch, refc=refc, remove_segments=remove_segments, 
                            force_make=True, force_make_raw=False, t1=t1, t2=t2, N=N, plotfolder=preprocessed_path, temp_path=temp_path, plotflag=True, 
                            stdmul=stdmul, fsds=fsds, notchflag=True, save_raw_flag=save_raw_flag)

# get preds

In [ ]:
importlib.reload(TRFs)
preds = TRFs.get_preds(preds, predspath=predictor_path, ks=['rect', 'oss', 'ossa', 'gt', 'zil'])

In [ ]:
importlib.reload(TRFs)
pred_shifts = TRFs.get_pred_shifts(preds)

# Fit TRFs

## All preds

In [ ]:
importlib.reload(TRFs)
options = [dict(cond='A', pndtype='true', pndsm=0.3, nbins=4, ks=['rect', 'oss', 'ossa', 'zil'], signs=['p'])]
TRFs.fit_TRFs(preds, options=options)

## slow change only

In [ ]:
importlib.reload(TRFs)
options = [dict(cond='fixed1', pndtype='true', pndsm=0.3, nbins=4, ks=['gt'], signs=['p'])]
TRFs.fit_TRFs(preds, options=options)

## fast change only

In [ ]:
importlib.reload(TRFs)
options = [dict(cond='switch1', pndtype='true', pndsm=0.3, nbins=4, ks=['gt'], signs=['p'])]
TRFs.fit_TRFs(preds, options=options)

## datalength

In [ ]:
importlib.reload(TRFs)
for Ntrials in range(36, 41, 4):
    options = [dict(cond='switch', pndtype='true', pndsm=0.3, nbins=4, ks=['gt'], signs=['p'], Ntrials=Ntrials)]
    TRFs.fit_TRFs(preds, options=options)

## Inherent

In [ ]:
importlib.reload(TRFs)
options = [dict(cond='A', pndtype='pred', pndsm=0.3, nbins=8, ks=['gt'], signs=['p'])]
TRFs.fit_TRFs(preds, options=options, subjects=[f'TP00{i}' for i in range(13, 25)])